# Introduction

The U-Net model is a simple fully  convolutional neural network that is used for binary segmentation i.e foreground and background pixel-wise classification. Mainly, it consists of two parts. 

*   Encoder: we apply a series of conv layers and downsampling layers  (max-pooling) layers to reduce the spatial size 
*   Decoder: we apply a series of upsampling layers to reconstruct the spatial size of the input. 

The two parts are connected using a concatenation layers among different levels. This allows learning different features at different levels. At the end we have a simple conv 1x1 layer to reduce the number of channels to 1.

Find the original paper [here](https://arxiv.org/abs/1505.04597).

![](https://drive.google.com/uc?export=view&id=1eKe1hMcn_kRc1xaakIYkFmHlmi9SKUbR)

This notebook is full of holes - it is intended to be filled by a user as a hands-on practice. To see one of possible versions with outputs, see <https://colab.research.google.com/drive/1tuTJ6vzNzsacebWkK-OH7WvcHoh3k8Hb>. 

Important - do a copy of that notebook and modify your own copy, otherwise you destroy it for all the other users.

# Download and organize the training dataset

Data is going to be downloaded using `odse_dl` Python package.

## Install `odse_dl` requirements

In [ ]:
!pip install -q geopandas rasterio

## Install `odse_dl`

In [ ]:
!pip install -q "git+https://gitlab.com/geoharmonizer_inea/odse-workshop-2022.git#subdirectory=python_training/packages/odse_dl"

## Download the dataset

In [ ]:
from odse_dl import data

Check what data can the package download

In [ ]:
data.sentinel_urls

In [ ]:
# according to the documentation, we can download them with the following:
# files = data.get_sentinel_tiles()

# but hey, they are soooo many. I'm gonna download only the summertime ones for this demo:
urls = [*filter(lambda url: '06.25..' in url and any(band in url for band in ('red', 'green', 'blue', 'nir')), data.sentinel_urls)]
files = data.input_data_to_tiles(urls)

Now let's organize them in some nice order. E.g.:

```
path/to/my/dataset/
├── train_images
│   ├── image_0.tif
│   ├── image_1.tif
│   ├── image_2.tif
│   └── image_4.tif
├── train_masks
│   ├── image_0.tif
│   ├── image_1.tif
│   ├── image_2.tif
│   └── image_4.tif
├── val_images
│   └── image_3.tif
└── val_masks
    └── image_3.tif
```

In [ ]:
import os

dir_paths = ('training_set', 'training_set/train_images', 
             'training_set/train_masks', 'training_set/train_masks', 
             'training_set/val_images', 'training_set/val_masks')

for dir_path in dir_paths:
    if not os.path.isdir(dir_path):
        os.mkdir(dir_path)

Let's copy the labels from `odse_dl.data` and make stacks of individual bands with `GDAL`

In [ ]:
# copy labels (target data)
!for i in 1 3; do cp /usr/local/lib/python3.7/dist-packages/odse_dl/dist_data/target_t${i}.tif training_set/train_masks/t${i}.tif; done
!for i in 4; do cp /usr/local/lib/python3.7/dist-packages/odse_dl/dist_data/target_t${i}.tif training_set/val_masks/t${i}.tif; done

# create stacks of Sentinel images (using here only RGB + NIR, but feel free to be more brave)
!for i in 1 3; do gdal_merge.py -separate  -o training_set/train_images/t${i}.tif -co PHOTOMETRIC=MINISBLACK input_data/lcv_red_sentinel.s2l2a_p50_30m_0..0cm_2018.06.25..2018.09.12_eumap_epsg3035_v1.0_t${i}.tif input_data/lcv_green_sentinel.s2l2a_p50_30m_0..0cm_2018.06.25..2018.09.12_eumap_epsg3035_v1.0_t${i}.tif input_data/lcv_blue_sentinel.s2l2a_p50_30m_0..0cm_2018.06.25..2018.09.12_eumap_epsg3035_v1.0_t${i}.tif input_data/lcv_nir_sentinel.s2l2a_p50_30m_0..0cm_2018.06.25..2018.09.12_eumap_epsg3035_v1.0_t${i}.tif; done
!for i in 4; do gdal_merge.py -separate  -o training_set/val_images/t${i}.tif -co PHOTOMETRIC=MINISBLACK input_data/lcv_red_sentinel.s2l2a_p50_30m_0..0cm_2018.06.25..2018.09.12_eumap_epsg3035_v1.0_t${i}.tif input_data/lcv_green_sentinel.s2l2a_p50_30m_0..0cm_2018.06.25..2018.09.12_eumap_epsg3035_v1.0_t${i}.tif input_data/lcv_blue_sentinel.s2l2a_p50_30m_0..0cm_2018.06.25..2018.09.12_eumap_epsg3035_v1.0_t${i}.tif input_data/lcv_nir_sentinel.s2l2a_p50_30m_0..0cm_2018.06.25..2018.09.12_eumap_epsg3035_v1.0_t${i}.tif; done

# Let's write the utils we are gonna need for the model itself

Import a million of packages and modules we are going to use.

In [ ]:
import cv2

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from odse_dl import legend
from osgeo import gdal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Concatenate, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras import backend as K

## Generators

We need a data generator - a [generator](https://wiki.python.org/moin/Generators) object yielding image data from a specified directory in the needed form (a batch of `numpy` arrays). [Onehot encoding](https://www.kaggle.com/code/dansbecker/using-categorical-data-with-one-hot-encoding/notebook) is performed to help the training accuracy.

In [ ]:
class AugmentGenerator:
    """Data generator."""

    def __init__(self, data_dir, batch_size=5, operation='train',
                 onehot_encode=True):
        """Initialize the generator.

        :param data_dir: path to the directory containing images
        :param batch_size: the number of samples that will be propagated
            through the network at once
        :param operation: either 'train' or 'val'
        :param onehot_encode: boolean to onehot-encode masks during training
        """
        images_dir = os.path.join(
            data_dir, '{}_images'.format(operation))
        masks_dir = os.path.join(
            data_dir, '{}_masks'.format(operation))

        # create variables useful throughout the entire class
        self.batch_size = batch_size
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.perform_onehot_encoding = onehot_encode

    def __call__(self):
        """Generate batches of data.

        :return: yielded tuple of batch-sized np stacks of validation images
            and masks
        """
        return self.generate_numpy()

    def generate_numpy(self):
        """Generate batches of data using our own numpy generator.

        Note: tf.data.Dataset.from_generator() seemed to be useful and maybe
        could speed up the process little bit , but it seemed not to work
        properly when __call__ takes arguments.

        :return: yielded tuple of batch-sized np stacks of validation images
            and masks
        """
        # create generators
        image_generator = self.numpy_generator(
            self.images_dir, self.batch_size)
        mask_generator = self.numpy_generator(
            self.masks_dir, self.batch_size)

        while True:
            x1i = next(image_generator)
            x2i = next(mask_generator)
            x2i = legend.transform(
                {0: 0, 1: 0, 2: 0, 3: 1, 4: 0, 5: 0, 6: 0, 7: 1, 8: 1}, x2i)
            # x2i = legend.transform(
            #     {0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 5: 1, 6: 4, 7: 2, 8: 2}, x2i)  # 3 plus aggr
            # x2i = legend.transform(
            #     {0: 0, 1: 1, 2: 1, 3: 2, 4: 3, 5: 1, 6: 2, 7: 2, 8: 2}, x2i)  # 3 classes

            if self.perform_onehot_encoding is True:
                # one hot encode masks
                x2i = [
                    self.onehot_encode(x2i[x, :, :, :]) for x in
                    range(x2i.shape[0])]

            yield x1i, np.asarray(x2i)

    def numpy_generator(self, data_dir, batch_size=5):
        """Generate batches of images.

        :param data_dir: path to the directory containing images
        :param batch_size: the number of samples that will be propagated
            through the network at once
        :return: yielded batch-sized np stack of images
        """
        # list of files from which the batches will be created
        source_list = sorted(os.listdir(data_dir))

        index = 1
        batch = []

        while True:
            for source in source_list:
                image = self.transpose_image(data_dir, source)

                # add the image to the batch
                batch.append(image)

                if index % batch_size == 0:
                    # batch created, return it
                    yield np.stack(batch)
                    batch = []

                index += 1

    def get_transposed_images(self, data_dir):
        """Get a list of transposed images.

        :param data_dir: path to the directory containing images
        :return: list of transposed numpy matrices representing images in
            the dataset
        """
        # list of files from which the dataset will be created
        files_list = sorted(os.listdir(data_dir))

        images_list = [
            self.transpose_image(data_dir, file) for file in
            files_list]

        return images_list

    @staticmethod
    def transpose_image(data_dir, image_name):
        """Open an image and transpose it to (1, 2, 0).

        :param data_dir: path to the directory containing images
        :param image_name: name of the image file in the data dir
        :return: the transposed image as a numpy array
        """
        image = gdal.Open(os.path.join(data_dir, image_name), gdal.GA_ReadOnly)
        image_array = image.ReadAsArray()

        # GDAL reads masks as having no third dimension
        # (we want it to be equal to one)
        if image_array.ndim == 2:
            transposed = np.expand_dims(image_array, -1)
        else:
            # move the batch to be the last dimension
            transposed = np.moveaxis(image.ReadAsArray(), 0, -1)

        image = None

        return transposed

    @staticmethod
    def onehot_encode(orig_image):
        """Encode input images into one hot ones.

        Unfortunately, keras.utils.to_categorical cannot be used because our
        classes are not consecutive.

        :param orig_image: original image
            (height x width x num_classes)
        """
        unique_vals = np.unique(orig_image)
        num_classes = len(unique_vals)
        shape = orig_image.shape[:2] + (num_classes,)
        encoded_image = np.empty(shape, dtype=np.uint8)

        # reshape to the shape used inside the onehot matrix
        reshaped = orig_image.reshape((-1, 1))

        for i, _ in enumerate(np.unique(reshaped)):
            all_ax = np.all(reshaped == i, axis=1)
            encoded_image[:, :, i] = all_ax.reshape(shape[:2])

        return encoded_image


def onehot_decode(onehot, nr_bands=3, enhance_colours=True):
    """Decode onehot mask labels to an eye-readable image.

    :param onehot: one hot encoded image matrix (height x width x
        num_classes)
    :param colormap: dictionary mapping label ids to their codes
    :param nr_bands: number of bands of intended input images
    :param enhance_colours: Enhance the contrast between colours
        (pseudorandom multiplication of the colour value)
    :return: decoded RGB image (height x width x 3)
    """
    # create 2D matrix with label ids (so you do not have to loop)
    single_layer = np.argmax(onehot, axis=-1)

    # create colourful visualizations
    out_shape = (onehot.shape[0], onehot.shape[1], nr_bands)
    output = np.zeros(out_shape)
    for k in range(onehot.shape[2]):
        output[single_layer == k] = k

    if enhance_colours is True:
        multiply_vector = [i ** 3 for i in range(1, nr_bands + 1)]
        enhancement_matrix = np.ones(out_shape) * np.array(multiply_vector,
                                                           dtype=np.uint8)
        output *= enhancement_matrix

    return np.uint8(output)

Let's have a `train` and a `val` (validation) generator.

Let's take a look at the data. Just to check if it works.

That looks actually terrible - but reality is for losers. Let's open the gates of perception and enhance the brightness.

In [ ]:
# how to increase brightness?
# -> put "python increase brightness" to Google
# copy the first solution on stackoverflow: https://stackoverflow.com/questions/32609098/how-to-fast-change-image-brightness-with-python-opencv
def increase_brightness(img, value=30):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

# Metrics

We are going to write our own metrics

In [ ]:
def mean_iou(y_true, y_pred):
    yt0 = y_true[:,:,:,0]
    yp0 = K.cast(y_pred[:,:,:,0] > 0.5, 'float32')
    inter = tf.math.count_nonzero(tf.logical_and(tf.equal(yt0, 1), tf.equal(yp0, 1)))
    union = tf.math.count_nonzero(tf.add(yt0, yp0))
    iou = tf.where(tf.equal(union, 0), 1., tf.cast(inter/union, 'float32'))
    return iou

def categorical_dice(ground_truth_onehot, predictions, weights=1):
    """Compute the Sorensen-Dice loss.

    :param ground_truth_onehot: onehot ground truth labels
        (batch_size, img_height, img_width, nr_classes)
    :param predictions: predictions from the last layer of the CNN
        (batch_size, img_height, img_width, nr_classes)
    :param weights: weights for individual classes
        (number-of-classes-long vector)
    :return: dice loss value averaged for all classes
    """
    loss = categorical_tversky(ground_truth_onehot, predictions, 0.5, 0.5,
                               weights)

    return loss


def categorical_tversky(ground_truth_onehot, predictions, alpha=0.5,
                        beta=0.5, weights=1):
    """Compute the Tversky loss.

    alpha == beta == 0.5 -> Dice loss
    alpha == beta == 1 -> Tanimoto coefficient/loss

    :param ground_truth_onehot: onehot ground truth labels
        (batch_size, img_height, img_width, nr_classes)
    :param predictions: predictions from the last layer of the CNN
        (batch_size, img_height, img_width, nr_classes)
    :param alpha: magnitude of penalties for false positives
    :param beta: magnitude of penalties for false negatives
    :param weights: weights for individual classes
        (number-of-classes-long vector)
    :return: dice loss value averaged for all classes
    """
    weight_tensor = tf.constant(weights, dtype=tf.float32)
    predictions = tf.cast(predictions, tf.float32)
    ground_truth_onehot = tf.cast(ground_truth_onehot, tf.float32)

    # compute true positives, false negatives and false positives
    true_pos = ground_truth_onehot * predictions
    false_neg = ground_truth_onehot * (1. - predictions)
    false_pos = (1. - ground_truth_onehot) * predictions

    # compute Tversky coefficient
    numerator = true_pos
    numerator = tf.reduce_sum(numerator, axis=(1, 2))
    denominator = true_pos + alpha * false_neg + beta * false_pos
    denominator = tf.reduce_sum(denominator, axis=(1, 2))
    tversky = numerator / denominator

    # reduce mean for batches
    tversky = tf.reduce_mean(tversky, axis=0)

    # reduce mean for classes and multiply them by weights
    loss = 1 - tf.reduce_mean(weight_tensor * tversky)

    return loss

# Model

Let's write our own U-Net. A simple way.

In [ ]:
tf.random.set_seed(437294792)
# tf.random.set_seed(2)



In [ ]:
model = unet()

Let's take a look what the model looks like

In [ ]:
model.summary()

# Training

Okay, let's start the fun.

# Callbacks

Fancy stuff to do at the end of each epoch

# Testing

In [ ]:
x, y = next(val_generator)
        
# get the prediction
pred = model.predict(x)

# prediction post-processing for the sake of visualization
pred_squeezed  = pred.squeeze()
# normalize the image and make it an integer one
img = (x[0][:, :, :3] * 255 / np.max(x[0][:, :, :3])).astype(np.uint8)
img = increase_brightness(img, value=60)
y_decode = onehot_decode(y[0])
mask = onehot_decode(pred_squeezed)

#show the mask and the prediction
combined = np.concatenate([img, y_decode * 8, mask * 8], axis=1)

plt.figure(figsize=(20, 20))
plt.axis('off')
plt.imshow(combined)
plt.show()